# **This notebook uses Content_based Model with feature weights to evaluate the model by Amazon evalaution data**

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
!pip install rank-bm25
from rank_bm25 import BM25Okapi
import math
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import pickle
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#change directory to current location
%cd /content/drive/My Drive/DMC

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/DMC


In [ ]:
# read in dataset
with open('publisher_author_cluster_added_new.csv', 'rb') as file:
    df = pickle.load(file)
    
ev =pd.read_csv('dmc21_amazon_validation.csv')
ev['amazon_list'] = [list() for x in range(len(ev.index))]

In [ ]:
max_num = 7    # maximum number of recommendations
for idx,row in ev.iterrows():
    for i in range(1, max_num + 1):
        if pd.isnull(row[i])== False and len(row[i]) <= 5:
            row['amazon_list'].append(row[i])
ev=ev.drop(columns=['rec1_ID','rec2_ID','rec3_ID','rec4_ID','rec5_ID','rec6_ID','rec7_ID'])
display(ev.head())

,itemID,amazon_list
0,55501,[17214]
1,50338,[34534]
2,23159,"[76833, 58623, 28475, 75884, 69228, 20239]"
3,13382,"[8193, 52254, 78733, 78225, 20674]"
4,53793,"[63382, 34773, 76742, 17001, 23012]"


In [ ]:
'''
eval_data = pd.read_csv('evaluation.csv')
eval_data['predictions'] = [list() for x in range(len(eval_data.index))] 
display(eval_data.head(1))'''

,itemID,predictions
0,12,[]


In [ ]:
# merge with the preprocessed items.csv to get features
evaluation_new = pd.merge(ev,df, how='left', on='itemID')
evaluation_new['predictions'] = [list() for x in range(len(evaluation_new.index))] 
print(evaluation_new.shape)
display(evaluation_new.head(1))

(232, 18)


,itemID,amazon_list,title,author,publisher,main topic,subtopics,language,interest_age,description,publisher_cluster,author_cluster,number_pages,maintopic_text,subtopics_text,release_date,processed_description,predictions
0,55501,[17214],The Royal Ranger (Ranger's Apprentice Book 12),John (Author) Flanagan,Penguin Random House Children's UK,YFC,"[5AK,YFH]",en,1.0,John Flanagan makes a spectacular return to th...,1,6.0,352.0,Children’s / Teenage fiction: Action and adven...,"['Interest age: from c 9 years', 'Children’s /...",{Timestamp('2021-10-05 00:00:00')},john flanagan make spectacular return world ra...,[]


### Model Building

In [ ]:
def searchquery(df, itemID, num_recommendation = 5):
    #Find the query item
    query = df.loc[df['itemID'] == itemID]
    return query

In [ ]:
#function required for title simililarity calculation
def calculate_bm25(query, corpus):
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    return doc_scores

In [ ]:
#The Linsimilarityfunction gets two strings (two main topics), Based on Lin similarity formula, then a floating number will
#return as a measure for similarity of two books.
def linsimilarityfunction (maintopic1 , maintopic2):
    #if two topics are identical the Lin similarity measure should be 1:
    if (maintopic1 == maintopic2):
        return 1
    #if they are not identical, we should do further analysis.
    #we compare the first elements of both lists. If they differ from each other, the Lin similarity will be zero.
    elif (maintopic1[0] != maintopic2[0]):
        return 0
    #otherwise we need further steps!
    else:
        #here we simply find the frequency of each topic in our dataset as it is required by Lin similarity
        totalitems = df['main topic'].count()
        minlength=min(len(maintopic1),len(maintopic2))
        df1=df[df['main topic']==maintopic1]
        df2=df[df['main topic']==maintopic2]
        count1=df1['main topic'].count()
        count2=df2['main topic'].count()
        #Now, we can focus on finding the closest mutual ancestor of both topics. By close, we refer to number of
        #node traverse in the tree of all topics.
        i=minlength
        while (i>0):
            if (maintopic1[:i]==maintopic2[:i]):
                dfmutual=df[df['main topic'].str[:i]==maintopic1[:i]]
                countmutual=dfmutual['main topic'].count()
                sim = 2*math.log10(countmutual/totalitems)/(math.log10(count1/totalitems)+math.log10(count2/totalitems))
                return sim
            else:
                i=i-1
                
# example
print(linsimilarityfunction ('YB', 'YFB'))

0.10116165919446315


In [ ]:
#preprocessing for interest age
#if interest_age = 'None', we assigned the item value 20 (as one of the member in Group others)
df['interest_age'] = [20 if i == 'None' else i for i in df['interest_age']]

#change data type from string to float
df['interest_age'] = df['interest_age'].astype(float)

In [ ]:
#we simply define the cosine similarity marices for controids of publisher clusters, according to what we calculated before.
#we will use these matrices, later in this notebook.
cosine_sim_pub_en= [[1.0, 0.8330066502646305,0.6297570399979626,0.4861936761165772,0.7453411410708859],
                    [0.8330066502646305, 1.0, 0.7554188417173869, 0.7118815933864038, 0.9210669994486579],
                    [0.6297570399979626, 0.7554188417173869, 1.0, 0.6235440834287432, 0.6375244958925325],
                    [0.4861936761165772, 0.7118815933864038, 0.6235440834287432, 1.0, 0.6478558391290395],
                    [0.7453411410708859, 0.9210669994486579, 0.6375244958925325, 0.6478558391290395, 1.0]]
cosine_sim_pub_de= [[1.0, 0.6980035928496665, 0.6652253308854318, 0.7618509407590829, 0.24346291180752616],
                    [0.6980035928496665, 1.0, 0.5094143512344115, 0.6684881408627634, 0.34110548671081775],
                    [0.6652253308854318, 0.5094143512344115, 1.0, 0.5009705224224626, 0.1478384568079113],
                    [0.7618509407590829, 0.6684881408627634, 0.5009705224224626, 1.0, 0.21427313660101593],
                    [0.24346291180752616, 0.34110548671081775, 0.1478384568079113, 0.21427313660101593, 1.0]]
cosine_sim_pub_es= [[1.0000000000000002, 0.160890554358734, 0.6464809047245317, 0.2896861543901769, 0.21631925136172767],
                    [0.160890554358734, 1.0, 0.15004107600273364, 0.15892212224517466, 0.08412392024908043], 
                    [0.6464809047245317, 0.15004107600273364, 1.0, 0.260231004956859, 0.35929479439712453], 
                    [0.2896861543901769, 0.15892212224517466, 0.260231004956859, 1.0000000000000002, 0.10176734676342591],
                    [0.21631925136172767, 0.08412392024908043, 0.35929479439712453, 0.10176734676342591, 1.0000000000000002]]
cosine_sim_pub_it= [[0.9999999999999999, 0.2561779725373971, 0.25311718656092913, 0.22072994629183765, 0.21016282367168967],
                    [0.2561779725373971, 1.0000000000000002, 0.2929791491742338, 0.060380627840515635, 0.10947151505277775],
                    [0.25311718656092913, 0.2929791491742338, 1.0000000000000002, 0.0947600541304488, 0.06665679929131706],
                    [0.22072994629183765, 0.060380627840515635, 0.0947600541304488, 1.0, 0.08102817035798306],
                    [0.21016282367168967, 0.10947151505277775, 0.06665679929131706, 0.08102817035798306, 1.0]]
cosine_sim_pub_fr= [[1.0, 0.0648745051530169, 0.10092346226021406, 0.12995150974842662,  0.11242176980768166],
                    [0.0648745051530169, 0.9999999999999998, 0.11951700602218226, 0.1598015065807482, 0.11855046701545281],
                    [0.10092346226021406, 0.11951700602218226, 1.0000000000000002, 0.4525789852904426, 0.45974607409831275],
                    [0.12995150974842662, 0.1598015065807482, 0.4525789852904426, 1.0000000000000002, 0.6061514610077626],
                    [0.11242176980768166, 0.11855046701545281, 0.45974607409831275, 0.6061514610077626, 0.9999999999999998]]

In [ ]:
#change data type from string to integer
df['publisher_cluster'] = df['publisher_cluster'].astype(int)

In [ ]:
#function for publisher cluster similarity calculation. 

def calculate_pub_cluster_similarity (publisher_clusster_i, publisher_clusster_j, language):
    if (language == 'en'):
        return cosine_sim_pub_en [publisher_clusster_i][publisher_clusster_j]
    elif (language == 'de'):
        return cosine_sim_pub_de [publisher_clusster_i][publisher_clusster_j]
    elif (language == 'es'):
        return cosine_sim_pub_es [publisher_clusster_i][publisher_clusster_j]
    elif (language == 'it'):
        return cosine_sim_pub_it [publisher_clusster_i][publisher_clusster_j]
    elif (language == 'fr'):
        return cosine_sim_pub_fr [publisher_clusster_i][publisher_clusster_j]
    else:
        return 1

In [ ]:
#required preprocessing for author cluster feature. 
#Also we import the cosine similarity matrix that we have computed for this feature.

df1 = df[df['author'].notnull()]
df2 = df[df['author'].isna()]

#if author = NaN we assigned the item value 20 as a number for author cluster attribute. it represent unknown! :D
df2['author_cluster'] = 20

frames = [df1, df2]
df = pd.concat(frames)

#change data type from string to integer
df['author_cluster'] = df['author_cluster'].astype(int)


#we calculated cosine similarity of author cluster centroids, and we just import the matrix save in a nparray
from numpy import load
# load array
cosine_sim_author = load('cosine_sim_author.npy', allow_pickle=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#function for author cluster similarity calculation. 
def calculate_author_cluster_similarity (author_cluster_i, author_cluster_j):
    if ((author_cluster_i == 20) or (author_cluster_j == 20)):
        return 0.5
    else:
        return cosine_sim_author [author_cluster_i][author_cluster_j]

In [ ]:
#required preprocessing for number of pages feature. 

df1 = df[df['number_pages'].notnull()]
df2 = df[df['number_pages'].isna()]
#if number_pages = NaN we assigned the item value 0 as a number for number_pages attribute. it represent unknown! :D
df2['number_pages'] = 0
frames = [df1, df2]
df = pd.concat(frames)
#change data type from string to integer
df['author_cluster'] = df['author_cluster'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def calculate_num_pages_similarity (num_pages_i, num_pages_j):
    if (num_pages_i <= 24):
        num_pages_i = 24
    elif (num_pages_i >= 640):
        num_pages_i = 640
    if (num_pages_j <= 24):
        num_pages_j = 24
    elif (num_pages_j >= 640):
        num_pages_j = 640
    return  1-(abs(num_pages_i-num_pages_j)/(640-24))

In [ ]:
after_remove_age = []
after_remove_publisher_clustering = []

In [ ]:
def recommend(df, itemID, num_recommendation = 5):
    
    #Find the item index
    item_index = int(df[df['itemID'] == itemID].index.values)

    #metadata for the itemID
    item_metadata = df[df['itemID'] == itemID]
    item_metadata = item_metadata.squeeze(axis=0)
    
    #0. Language: Keep data with same langauge. + remove itemID (this is the query item) from df dataframe
    df = df.loc[df['itemID'] != itemID]
    df = df.loc[df['language'] == item_metadata.language]
    

    #1. Title
    item_title = item_metadata.title #this is the query sentence
    if pd.isnull(item_title)== False:
        df['title_bm25_scores'] = calculate_bm25(item_title, df.title)
    else: 
        df['title_bm25_scores'] = 0

    #2. Description
    item_description = item_metadata.processed_description #this is the query sentence
    if pd.isnull(item_description)== False:
        df['description_bm25_scores'] = calculate_bm25(item_description, df.processed_description)
    else: 
        df['description_bm25_scores'] = 0

    #3. Calculate the closeness of age between query item and candidate items. At the end, create an new column called "age_closeness"
    #age distance
    df['age_distance'] = df['interest_age'] - item_metadata['interest_age'] #candidate's age minus query item's age
    #age Normalization: use Max-min Normalization
    df['age_distance'] = (df['age_distance'] - df['age_distance'].min()) / (df['age_distance'].max() - df['age_distance'].min())
    #How close the age distance is? (Closeness = 1 - age_distance)
    df['age_closeness'] = 1 - abs(df['age_distance']) #the larger value the closer age is    

    #4. Publisher clusters. since we have different clusters for different languages: first, we capture the language of
    #both books. Then, we should look for each specific language. so, if both books are written in English, German, Spanish,
    #Italian, or French, then, we simply use the cosine similarity matrix that we extracted during the clustering.
    #Otherwise, we set the measure to 1, which means that for all infrequent languages, we basically use the same score
    #equal to 1 for all books.
    item_publisher_clusster = item_metadata['publisher_cluster']
    item_language = item_metadata['language']
    df['publisher_cluster_scores'] = [calculate_pub_cluster_similarity (item_publisher_clusster, publisher_cluster, item_language) for publisher_cluster in df['publisher_cluster']]
    
    #5. Title-Author combination best selections
    item_author = item_metadata.author
    df1 = df.loc[df['author'] == item_author]
    df2 = df.loc[df['author'] != item_author]
    df1['author_scores'] = 1
    df2['author_scores'] = 0

    #6. only keep age closeness larger than 0.95 (Q1)
    #print('\n')
    #print('The number of current data is: ', len(df)) 
    
    if (len(df2.loc[df2['age_closeness'] > 0.95]) < 1000):
      df2 = df2.sort_values(by=['age_closeness'], ascending=False)
      df2= df2[:1000]
      #print('The number of current data after removing age closeness lower than 0.95 is: ', len(df))
    else:
      df2 = df2.loc[df['age_closeness'] > 0.95]
      #print('The number of current data after removing age closeness lower than 0.95 is: ', len(df)) 
    
    after_remove_age.append(len(df2))


    #7. only keep publisher clustering larget than 0.76 (Q1)

    if (len(df2.loc[df2['publisher_cluster_scores'] > 0.76]) < 500):
      df2 = df2.sort_values(by=['publisher_cluster_scores'], ascending=False)
      df2= df2[:500]
      #print('The number of current data after removing age closeness lower than 0.95 is: ', len(df))
    else:
      df2 = df2.loc[df['publisher_cluster_scores'] > 0.76]
      #print('The number of current data after removing age closeness lower than 0.95 is: ', len(df)) 
    
    after_remove_publisher_clustering.append(len(df2))
       
 
    #8.combining all we have for final evaluation
    df2 = df2.sort_values(by=['description_bm25_scores'], ascending=False)
    if len(df2.index) >= 1000:
        df2= df2[:1000]
    frames = [df1, df2]
    df = pd.concat(frames)

    
    #9. author clusters
    item_author_clusster = item_metadata['author_cluster']
    df['author_cluster_scores'] = [calculate_author_cluster_similarity (item_author_clusster, author_cluster) for author_cluster in df['author_cluster']]

    
    #10. number of pages
    item_num_pages = item_metadata['number_pages']
    if (item_num_pages == 0):
        df['num_pages_scores'] = 0
    else:
        df1 = df[df['number_pages']!= 0]
        df2 = df[df['number_pages']== 0]
        df1['num_pages_scores'] = [calculate_num_pages_similarity (item_num_pages, number_pages) for number_pages in df1['number_pages']]
        df2['num_pages_scores'] = 0
        frames = [df1, df2]
        df = pd.concat(frames)
    
    #11. Main topic: use Linsimilarity
    item_maintopic = item_metadata['main topic']
    if pd.isnull(item_maintopic)==False:
        df1 = df[df['main topic'].notna()]
        df2 = df[df['main topic'].isna()]
        df1['maintopic_scores'] = [linsimilarityfunction (item_maintopic, topic) for topic in df1['main topic']]
        df2['maintopic_scores'] = 0
        #combining all we have for final evaluation
        frames = [df1, df2]
        df = pd.concat(frames)
    else:
        df['maintopic_scores'] = 0
    
    #normalize all scores.
    df['maintopic_scores'] = (df['maintopic_scores'] - df['maintopic_scores'].min()) / (df['maintopic_scores'].max() - df['maintopic_scores'].min())
    Total = df['title_bm25_scores'].sum()
    if (Total > 0):
        df['title_bm25_scores'] = (df['title_bm25_scores'] - df['title_bm25_scores'].min()) / (df['title_bm25_scores'].max() - df['title_bm25_scores'].min())   
  
    
    
    #12. final score and limit the number of recommendations to 10
    
    #since we had many NaN values in the num_pages column, we should find a way to handle the issue.
    #thus, for all items with NaN values (previously replacced by zero), we omit this criterion from our evaluation.
    if (item_num_pages == 0):
        df['average_score'] = 0.22265 *  df['author_scores'] + 0.27749 * df['maintopic_scores'] + 0.08067 * df['age_closeness'] + 0.07591 * df['title_bm25_scores'] + 0.03389 * df['publisher_cluster_scores'] + 0.30938 * df['author_cluster_scores']
    else:
        df1 = df[df['number_pages']!= 0]
        df2 = df[df['number_pages']== 0]
        df1['average_score'] = 0.10731 *  df1['author_scores'] + 0.12744 * df1['maintopic_scores'] + 0.04142 * df1['age_closeness'] + 0.03289 * df1['title_bm25_scores'] + 0.01880 * df1['publisher_cluster_scores'] + 0.19771 * df1['author_cluster_scores'] + 0.47442 * df1['num_pages_scores']
        df2['average_score'] = 0.22265 *  df2['author_scores'] + 0.27749 * df2['maintopic_scores'] + 0.08067 * df2['age_closeness'] + 0.07591 * df2['title_bm25_scores'] + 0.03389 * df2['publisher_cluster_scores'] + 0.30938 * df2['author_cluster_scores']
        frames = [df1, df2]
        df = pd.concat(frames)

    df = df.drop_duplicates(subset=['itemID'], keep='first')
    df = df.drop_duplicates(subset=['title'], keep='first')
    result = df.sort_values(by=['average_score'], ascending=False)
    #result.drop(columns=['level_0','maintopic_text','subtopics_text','release_date', 'processed_description', 'age_distance'], axis= 1, inplace=True)
    result.drop(columns=['maintopic_text','subtopics_text','release_date', 'processed_description', 'age_distance'], axis= 1, inplace=True)
    return result.iloc[:num_recommendation]


### **An Example for model test**

In [ ]:
query_item = searchquery(df, 13382, 5) #k = 5 #445
query_item.head()

,itemID,title,author,publisher,main topic,subtopics,language,interest_age,description,publisher_cluster,author_cluster,number_pages,maintopic_text,subtopics_text,release_date,processed_description
75521,13382,Die Macht der Drei,Hans Dominik,Hofenberg,FL,[],de,20.0,Die Macht der Drei ist ein technisch-wissensch...,1,3,384.0,Science fiction,NaN,{Timestamp('2022-01-27 00:00:00')},macht drei technisch wissenschaftlich zukunfts...


In [ ]:
%%time
rec_books = recommend(df, 13382, 5) #k = 5
display(rec_books.head())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,itemID,title,author,publisher,main topic,subtopics,language,interest_age,description,publisher_cluster,author_cluster,number_pages,title_bm25_scores,description_bm25_scores,age_closeness,publisher_cluster_scores,author_scores,author_cluster_scores,num_pages_scores,maintopic_scores,average_score
6963,4787,Das stählerne Geheimnis,Hans Dominik,Projekt Gutenberg,FL,[],de,1.0,Für RUTHeBooks Klassiker lassen wir alte oder ...,1,3,0.0,0.000000,5.453658,0.95,1.000000,1,1.0,0.000000,1.000000,0.920046
75520,46822,Die Macht der Drei,Hans Dominik,Books on Demand,FLC,[3MPBG],de,20.0,Die Macht der Drei ist ein technisch-wissensch...,3,3,384.0,1.000000,502.096164,0.00,0.668488,1,1.0,1.000000,0.560347,0.896308
75531,40521,Atlantis,Hans Dominik,Projekt Gutenberg,FL,[],de,20.0,"Why is Goti, the shiny red bus, teasing poor V...",1,3,320.0,0.000000,2.612285,0.00,1.000000,1,1.0,0.896104,1.000000,0.876390
56780,20674,Der Wettflug der Nationen,Hans Dominik,Projekt Gutenberg,FL,[],de,1.0,Ghost stories from America's Great Lakes State...,1,3,258.0,0.129897,0.000000,0.95,1.000000,1,1.0,0.795455,1.000000,0.872261
66888,69525,Die Nahrung der Zukunft (Großdruck),Hans Dominik,Henricus,FL,[],de,1.0,hans dominik die nahrung der zukunft acht klas...,1,3,244.0,0.229190,19.016767,0.95,1.000000,1,1.0,0.772727,1.000000,0.864744


CPU times: user 3.54 s, sys: 102 ms, total: 3.64 s
Wall time: 3.65 s


In [ ]:
query_item = searchquery(df, 12564, 5) #k = 5
query_item.head()

,itemID,title,author,publisher,main topic,subtopics,language,interest_age,description,publisher_cluster,author_cluster,number_pages,maintopic_text,subtopics_text,release_date,processed_description
18455,12564,Ignite Me,Tahereh Mafi,Egmont UK Limited,YFCB,"[5AQ,YFG,YFHR,YFM]",en,1.5,X-Men meets The Handmaid's Tale in the third i...,1,10,416.0,Children’s / Teenage fiction: Thrillers,"['Interest age: from c 14 years', 'Children’s ...",{Timestamp('2014-02-04 00:00:00')},x men meet handmaid tale third instal epic rom...


In [ ]:
rec_books = recommend(df, 12564, 5) #k = 5
display(rec_books)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,itemID,title,author,publisher,main topic,subtopics,language,interest_age,description,publisher_cluster,author_cluster,number_pages,title_bm25_scores,description_bm25_scores,age_closeness,publisher_cluster_scores,author_scores,author_cluster_scores,num_pages_scores,maintopic_scores,average_score
18457,9024,Imagine Me,Tahereh Mafi,Egmont UK Limited,YFCB,"[YFG,YFHR,YFM]",en,1.000000,The book that all SHATTER ME fans have been wa...,1,10,464.0,1.0,278.238908,0.950000,1.0,1,1.0,0.922078,1.000000,0.960951
18452,50498,Defy Me,Tahereh Mafi,Egmont UK Limited,YFCB,"[5AN,5AQ,YFG,YFHR,YFM]",en,1.333333,The breath-taking and heart-pounding fifth ins...,1,10,368.0,1.0,180.957221,0.933333,1.0,1,1.0,0.922078,1.000000,0.960261
18448,21222,Restore Me,Tahereh Mafi,Harper Collins Publ. USA,YFHR,"[YFE,YFG]",en,1.000000,An instant New York Times bestseller! Juliette...,1,10,448.0,1.0,109.221401,0.950000,1.0,1,1.0,0.948052,0.172344,0.867797
18446,59588,Shatter Me,Tahereh Mafi,Harper Collins Publ. USA,YFE,"[5AK,YFG,YFM]",en,1.000000,X-Men meets The Handmaid's Tale in this first ...,1,10,448.0,1.0,267.161290,0.950000,1.0,1,1.0,0.948052,0.147314,0.864608
18445,68724,Unravel Me,Tahereh Mafi,Harper Collins Publ. USA,YFE,"[5AQ,YFG,YFM]",en,1.500000,The thrilling second installment in New York T...,1,10,480.0,1.0,159.555821,0.925000,1.0,1,1.0,0.896104,0.147314,0.838927


### Evaluate on validation data (232 data)

In [ ]:
%%time

# generate recommendations based on book's features
for idx,row in evaluation_new.iterrows():
    print(idx)
    item_data = row['itemID']
    try: 
        rec_list =recommend(df, item_data, 5)['itemID'].tolist()
        for x in rec_list:
            row['predictions'].append(str(x))
    except:
        print('unable to make recommendations for the {}th book'.format(idx))

0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

2


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


231


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1h 35min 29s, sys: 37.6 s, total: 1h 36min 7s
Wall time: 1h 36min 2s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

### Evaluation Metrics

In [ ]:
# export the dataset for rating the RS
evaluation_data=evaluation_new[['itemID','amazon_list','predictions']]
evaluation_data.to_csv('evaluation_data.csv')

evaluation_data[['recall_no_order','precision_no_order','f1_no_order','precision_order']] = 0
display(evaluation_data.head())

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,itemID,amazon_list,predictions,recall_no_order,precision_no_order,f1_no_order,precision_order
0,55501,[17214],"[73054, 9962, 64290, 6722, 28841]",0,0,0,0
1,50338,[34534],"[34534, 847, 52620, 54674, 42444]",0,0,0,0
2,23159,"[76833, 58623, 28475, 75884, 69228, 20239]","[46377, 65708, 69228, 21183, 75884]",0,0,0,0
3,13382,"[8193, 52254, 78733, 78225, 20674]","[4787, 46822, 40521, 20674, 69525]",0,0,0,0
4,53793,"[63382, 34773, 76742, 17001, 23012]","[59496, 44738, 57134, 28629, 35498]",0,0,0,0


#### 1) if the order of recommendations is not considered
<br>

<img align="left" width="270" height="270" src="https://miro.medium.com/max/718/1*_ozCUOjGs6bDRiyjEqNIuQ.png">
<br>
<br>
<br>
<br>
<img align="left" width="270" height="270" src="https://miro.medium.com/max/590/1*OzBFqkAKVCsSuX5mR7hZpQ.png">
<br>
<br>
<br>
<br>
<img align="left" width="200" height="200" src="https://miro.medium.com/max/584/1*rKvZWyPe2EhzRMmB6n6Ygw.png">

In [ ]:
def evaluation_no_order(evaluation_data):
    for idx,row in evaluation_data.iterrows():
        recall,precision,f1 = 0,0,0
    
        amazon_set = set(evaluation_data.iloc[idx,1])
        predicts_set = set(row['predictions'])
    
        # recall
        if len(amazon_set) != 0:
            recall = round(len(amazon_set.intersection(predicts_set))/len(amazon_set),3)
            evaluation_data.iloc[idx,3] = recall
        # precision
        if len(predicts_set) != 0:
            precision = round(len(amazon_set.intersection(predicts_set))/len(predicts_set),3)
            evaluation_data.iloc[idx,4] = precision
        # f1-score
        if recall != 0 and precision != 0:
            f1 = round(2/(1/recall + 1/precision),3)
            evaluation_data.iloc[idx,5] = f1

    avg_recall = round(evaluation_data['recall_no_order'].mean(),4)
    avg_precision = round(evaluation_data['precision_no_order'].mean(),4)
    avg_f1 = round(evaluation_data['f1_no_order'].mean(),4)
    print("average recall:", avg_recall)
    print("average precision:", avg_precision)
    print("average f1:", avg_f1)

#### 2) If the order of recommendations is considered
<br>
<img align="left" width="370" height="370" src="https://miro.medium.com/max/789/1*jAA3h5rdgI_tIfdt4uqQGA.png">
<br>
<br>
<br>
<br>
precision@j is the precision, when we provide the top j recommendations. relevant@j is 1 when the jᵗʰ recommendation is relevant and 0, otherwise. 

(note that the example given by the author on the reference page indicates that the (precision*relevance) is added from j=1 to j=number_of_recommendations (here: 5), which differs from the fomular).

In [ ]:
def evaluation_with_order(evaluation_data):
    for idx,row in evaluation_data.iterrows():
        precision,relevance,total = 0,0,0
    
        amazon_list = evaluation_data.iloc[idx,1]
        predicts_list = row['predictions']
        length_min = min(len(amazon_list), len(predicts_list))  
        length_max = max(len(amazon_list), len(predicts_list))
        amazon_set = set(amazon_list)

        for i in range(0, len(predicts_list)):
            if predicts_list[i] in amazon_set:
                relevance=1
            else:
                relevance=0
            precision = len(amazon_set.intersection(predicts_list[0:i+1]))/(i+1)
            total += relevance*precision
    
        if length_min != 0:
            precision_with_order = total/length_min
            evaluation_data.iloc[idx,6] = precision_with_order
        #else:
        #    print('unable to make recommendations for the {}th book'.format(idx))
            
    avg_precision_with_order = round(evaluation_data['precision_order'].mean(),4)
    print("average precision(ordered):", avg_precision_with_order)

In [ ]:
evaluation_no_order(evaluation_data)
evaluation_with_order(evaluation_data)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


average recall: 0.2495
average precision: 0.1672
average f1: 0.1823
average precision(ordered): 0.2073


In [ ]:
display(evaluation_data)
evaluation_data.to_csv('evaluation_data_evaluated.csv')

,itemID,amazon_list,predictions,recall_no_order,precision_no_order,f1_no_order,precision_order
0,55501,[17214],"[73054, 9962, 64290, 6722, 28841]",0.000,0.0,0.000,0.000000
1,50338,[34534],"[34534, 847, 52620, 54674, 42444]",1.000,0.2,0.333,1.000000
2,23159,"[76833, 58623, 28475, 75884, 69228, 20239]","[46377, 65708, 69228, 21183, 75884]",0.333,0.4,0.363,0.146667
3,13382,"[8193, 52254, 78733, 78225, 20674]","[4787, 46822, 40521, 20674, 69525]",0.200,0.2,0.200,0.050000
4,53793,"[63382, 34773, 76742, 17001, 23012]","[59496, 44738, 57134, 28629, 35498]",0.000,0.0,0.000,0.000000
...,...,...,...,...,...,...,...
227,55488,"[63666, 27153, 61837, 29054, 4686]","[4686, 61837, 29054, 53582, 53086]",0.600,0.6,0.600,0.600000
228,56300,[27173],"[48335, 52874, 27486, 23541, 5070]",0.000,0.0,0.000,0.000000
229,76762,"[71574, 64261]","[66650, 12978, 61178, 64261, 14024]",0.500,0.2,0.286,0.125000
230,45435,"[7636, 65819, 64817]","[44757, 15978, 37565, 18081, 38044]",0.000,0.0,0.000,0.000000
